# ANF DeepRobot #
## Mise en œuvre des techniques d’apprentissage (« deep learning ») pour la robotique ##
### TP 01 : Classification d'images ###

Imports

In [19]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

import threading
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import PIL.Image
import numpy as np

from imagenet_classes import imagenet_classes

In [2]:
def preprocess(image):
    mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Init caméra

In [3]:
camera = CSICamera(width=224, height=224)
camera.running = True

In [4]:
# create live preview
camera_widget = ipywidgets.Image()
#traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# image snap
image_widget = ipywidgets.Image()

Modèle pré-entrainé

In [5]:
device = torch.device('cuda')

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
#model.fc = torch.nn.Linear(512, len(dataset.categories))

model = model.to(device)

model = model.eval()

Capture flux caméra avec inférence en direct

In [36]:
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

#state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
#score_widgets = []
#for category in dataset.categories:
#    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
#    score_widgets.append(score_widget)
score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description='score', orientation='vertical')
binfer_widget = ipywidgets.Button(description='infer')

image = []

##def live(state_widget, model, camera, prediction_widget, score_widget):
#def live(model, camera, binfer_widget, prediction_widget, score_widget):
#    global image
#    #global dataset
#    #while state_widget.value == 'live':
#    image = camera.value
#    if binfer_widget.value == True:
#        print("TOTO")
#    #preprocessed = (image)
#    #output = model(preprocessed)
#    #output = F.softmaxpreprocess(output, dim=1).detach().cpu().numpy().flatten()
#    #category_index = output.argmax()
#    #prediction_widget.value = imagenet_classes[category_index]
#    #score_widgets[0].value = output.max()
#    #prediction_widget.value = dataset.categories[category_index]
#    #for i, score in enumerate(list(output)):
#    #    score_widgets[i].value = score

#def start_live():#(change):
#    #if change['new'] == 'live':
#    #execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
#    execute_thread = threading.Thread(target=live, args=(model, camera, binfer_widget, prediction_widget, score_widget))
#    execute_thread.start()

#state_widget.observe(start_live, names='value')
#start_live()

def update_image(change):
    global image
    image = change['new']
    camera_widget.value = bgr8_to_jpeg(image)
camera.observe(update_image, names='value')

def infer(c):
    image_widget.value = bgr8_to_jpeg(image)
    #prediction_widget.value = 'haha'
    #prediction_widget.value = "{}".format(image.shape)
    # convert to tensor
    preprocessed = preprocess(np.array(image))
    #prediction_widget.value = "{}".format(preprocessed.shape)
    output = model(preprocessed)
    #prediction_widget.value = "{}".format(output.shape)
    #output = F.softmaxpreprocess(output, dim=1).detach().cpu().numpy().flatten()
    output = F.softmax(output, dim=1)
    #prediction_widget.value = "{}".format(output.shape)
    category_index = torch.argmax(output).item() #output.argmax()
    score = torch.max(output).item()
    #prediction_widget.value = "{}".format(category_index)
    prediction_widget.value = "{}: {}".format(category_index, imagenet_classes[category_index])
    score_widget.value = score
binfer_widget.on_click(infer)


live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, binfer_widget]),
    ipywidgets.HBox([image_widget, score_widget, prediction_widget])
])

display(live_execution_widget)

In [12]:
camera.unobserve(update_image, names='value')

806


'sock'